# Geometric factor calculations using Monte Carlo simulation data 
By Andrei R. Hanu - <andrei.hanu@nasa.gov>

## Introduction

The geometric factor ($\bf{G}$, units of cm$^2$ sr) of a particle detector is analogous to the "collecting power" of an optical telescope and is a function of the surface area of the instrument and detection efficiency, which itself is a function of the charge, mass, and energy of the incident particles. If the detector's geometry factor is known, the spatial differential flux ($\bf{\Phi}$, units of particles cm$^{-2}$ sr$^{-1}$ sec$^{-1}$) incident on the detector can be calculated from the counting rate ($\bf{N_D}$, units of particles $sec^{-1}$). So very simply, the geometric factor is defined as the counts received by a detector per unit time divided by the spatial differential flux around the instrument.

$$ G = \frac{N_D}{J} $$

** Note: For radiation risk assesment calculations, the flux is typically measured in energy bins with units of particles cm$^{-2}$ sr$^{-1}$ sec$^{-1}$ MeV$^{-1}$ **

Tradiationally, a detector's geometric factor has usually been estimated by approximation, however, only a few analytical solutions exist for simple source and detector geometries - see Sullivan (1971). For more complicated detector geometries, including electronic coincidence/anti-coincidence logic, an analytical solution might be difficult, or even impossible, to derive in the closed form. In such cases, the Monte Carlo technique is employed to compute the geometrical factor, see Crannell et al. (1971), by surrounded the detector geometry with a spherical source and assuming an isotropic radiation environment. Although, you can also apply this technique to anisotropic radiation environments. 

## Isotropic Radiation



## Source Normalization

For isotropic radiation environments, the number of particles $\bf{N_R}$ traversing a sphere is determined by the integrating the spatial differential flux over the surface area of the sphere and solid angle.

$$ N_R = \int_{\Omega}\int_{S} JdSd\Omega $$

Where the differential surface area ($\bf{dS}$) and differential solid angle ($\bf{d\Omega}$) in spherical coordinates is:

$$ dS = R^2 sin \theta d\theta d\phi $$

$$ d\Omega = sin \theta d\theta d\phi $$

Working through the integral, we see that the number of particles $\bf{N_R}$ traversing a sphere is:

$$ N_R = 4 \pi^2 R^2 J $$

And it depends on:

- The radius (ie. bigger sphere == more particles crossing the sphere)
- The spatial differential flux (ie. more flux == more particles crossing the sphere)

Finally, by using Eq.1 and Eq. 5 the geometric factor for an isotropic radiation environments can be written as follows:

$$ G = \frac{N_D}{N_R}4 \pi^2 R^2 $$

Where:

- $N_D$ is the number of particles interacting with the detector model
- $N_R$ is the total number of simulated particles
- $R$ is the radius of the spherical source encircling the detector model

The associated standard deviation ($\bf{\sigma_G}$) is obtained from the binomial distribution and follows:

$$ \sigma_G = 4\pi^2R^2 \sqrt{\left(1-\frac{N_D}{N_R}\right)\frac{N_D}{N_R^2}} $$

## Monte Carlo method

To calculate the geometric factor of a detector using the Monte Carlo method, the following steps need to be taken:

- The detector geometry, or a simplified model, should be implemented in your favourite Monte Carlo particle transport  code (Geant4, MCNP, FLUKA, EGS, etc)

If an isotropic flux, typical of space-borne instruments, is assumed:

- The detector geometry is enveloped by a spherical source such that the spectral, spatial, and angular distribution of primary particles can be specified

- The primary particles must be uniformly distributed over the entire surface of spherical source
- The angular distribution for the emission of primary particles must follow a cosine-law distribution

## References

+ Sullivan, J. D. "Geometric factor and directional response of single and multi-element particle telescopes." Nuclear Instruments and methods 95.1 (1971): 5-11.

+ Crannell, C. J., and J. F. Ormes. "Geometrical-factor determination using a monte carlo approach." Nuclear Instruments and Methods 94.1 (1971): 179-183.

+ Sanderson, T. R., and D. E. Page. "Geometrical aspects of the performance of cosmic ray detector telescopes in non-isotropic particle distributions." Nuclear Instruments and Methods 104.3 (1972): 493-504.


In [1]:
# Matplotlib - 2D plotting library
from matplotlib import pyplot as plt
from matplotlib import rcParams

# Show matplotlib figures inline the notebook
%matplotlib inline

# Pandas - High-performance data analysis
import pandas

# Glob - Unix style pathname pattern expansion
# Used to aggregate folders and files into a single list so they can be iterated over
import glob

# Numpy - For handling of large, multi-dimensional arrays and matrices
import numpy as np

# Divide integers
from __future__ import division

##########################################################################################
# Setting rcParams for publication quality graphs
fig_width_pt = 246.0                    # Get this from LaTeX using \showthe\columnwidth
inches_per_pt = 1.0/72.27               # Convert pt to inch
golden_mean = (np.sqrt(5)-1.0)/2.0      # Aesthetic ratio
fig_width = fig_width_pt*inches_per_pt  # Width in inches
fig_height = fig_width*golden_mean      # Height in inches
fig_size =  [fig_width, fig_height]
# fig_size =  [7.3,4.2]
fig_size =  [7.3*1.25,4.2*1.25]
# fig_size =  [7.3*1.5,4.2*1.5]
# fig_size =  [7.3*2,4.2*2]
params = {'backend': 'pdf',
        'axes.labelsize': 10,
        'legend.fontsize': 10,
        'xtick.labelsize': 10,
        'ytick.labelsize': 10,
        'xtick.major.size': 7,
        'xtick.major.width': 1,
        'xtick.minor.size': 3.5,
        'xtick.minor.width': 1.25,
        'ytick.major.size': 7,
        'ytick.major.width': 1.25,
        'ytick.minor.size': 3.5,
        'ytick.minor.width': 1.25,
        'font.family': 'sans-serif',
        'font.sans-serif': 'Avant Garde',
        'font.size': 10,
        'text.usetex': True,
        'figure.figsize': fig_size}

# Update rcParams
rcParams.update(params)

In [ ]:
##########################################################################################
# Energy of incident particles (Electrons)
# InputEnergies = np.concatenate([np.linspace(1000,9000, num=9, endpoint=True, dtype = int),
#                 np.linspace(1250,9250, num=9, endpoint=True, dtype = int),
#                 np.linspace(1500,9500, num=9, endpoint=True, dtype = int),
#                 np.linspace(1750,9750, num=9, endpoint=True, dtype = int),
#                 np.linspace(10000,100000, num=10, endpoint=True, dtype = int),
#                 ]).ravel()

InputEnergies = np.logspace(3.0, 5.0, num=40).astype(int)

# Folder containing simulation data
DataFolder = '../Results/Scint_4mm_PV_1mm_Lid_1mm/Electrons/'

##########################################################################################
# Energy thresholds (in eV) for events that will be included in the analysis
Thr_TEPC = 100.
Thr_ACD = 100.

# Print a list of input electron energies
print "Input Electron Energies: ", InputEnergies
# for InputEnergy in InputEnergies[InputEnergies.argsort()]:
#     print InputEnergy/1E3, "\t"

Input Electron Energies:  [  1000   1125   1266   1425   1603   1804   2030   2285   2571   2894
   3257   3665   4124   4641   5223   5878   6614   7443   8376   9426
  10608  11937  13433  15117  17012  19144  21544  24244  27283  30702
  34551  38881  43754  49238  55410  62355  70170  78965  88862 100000]


In [ ]:
##########################################################################################
# Load the simulation data for each input energy and calculate the geometric factor
# If the data file has energy in form 1e+06, you can use the following shell script to
# change the name:
# for f in *; do mv "$f" "`echo $f | sed -e "s/e+06/000000/g"`"; done

# Pandas dataframe to hold the raw number of events from each simulation
Data = pandas.DataFrame(index = InputEnergies,
                        columns = ('Source Particles', 
                                   'Total',
                                   'Subthreshold',
                                   'Coinc', 
                                   'Anti-Coinc'))

# Geant4 Source Information
R = 7.75    # Source Radius

# Pandas dataframe of the calculate geometric factors
DataGF = pandas.DataFrame(index = InputEnergies,
                          columns = ('GF Total [cm^2 sr]', 
                                     'GF Total [cm^2 sr] Error',
                                     'GF Coinc [cm^2 sr]',
                                     'GF Coinc [cm^2 sr] Error',
                                     'GF Anti-Coinc [cm^2 sr]',
                                     'GF Anti-Coinc [cm^2 sr] Error'))

# Loop through the input energies to process the data
for InputEnergy in InputEnergies[InputEnergies.argsort()]:
    # Create data file template from current energy 
    DataFilename = 'e-_' + str(InputEnergy) + 'keV_Nr_*_ISO_nt_G4CNPTEPC_t*.csv*'
    
    # Statistics variables for each input energy data
    nR = 0
    nTotal = 0
    nCoinc = 0
    nAntiCoinc = 0
    nSubThr = 0
    
    ##########################################################################################
    # Process the simulation data files
    # Column 0 - Energy deposited in the TEPC sensitive volume [eV]
    # Column 1 - Track length of particles that entirely traverse the TEPC [mm]
    # Column 2 - Energy deposited in the plastic scintillator [eV]
    for dataFile in glob.glob(DataFolder + DataFilename):  
        # Readout CSV files
        try:
            dataFrame = pandas.read_csv(dataFile, compression='gzip', header=-1, skiprows=6, usecols=[0,1,2]).values
#             dataFrame = pandas.read_csv(dataFile, header=-1, skiprows=6, usecols=[0,1,2]).values
    
            # Extract the number of source particles from the data filename
            nR = int(dataFile.split('_')[-5])

            # Increment statistics
            nTotal += dataFrame[(dataFrame[:,0] > Thr_TEPC),0].size
            nCoinc += dataFrame[np.logical_and((dataFrame[:,2] > Thr_ACD),(dataFrame[:,0] > Thr_TEPC)),0].size
            nAntiCoinc += dataFrame[np.logical_and((dataFrame[:,2] <= Thr_ACD),(dataFrame[:,0] > Thr_TEPC)),0].size
            nSubThr += dataFrame[(dataFrame[:,0] <= Thr_TEPC),0].size
        except:
            pass
    
    print 'Finished processing data files for', InputEnergy, 'keV'
    
    # Append the statistics to the Data list
    Data.loc[InputEnergy] = [nR,nTotal,nSubThr,nCoinc,nAntiCoinc]
    
    # Calculate the geometric factors and append to the DataGF list
    try:
        DataGF.loc[InputEnergy] = [nTotal/nR*4*(np.pi**2)*(R**2),
                                   4*(np.pi**2)*(R**2)*np.sqrt((1-nTotal/nR)*nTotal/nR**2),
                                   nCoinc/nR*4*(np.pi**2)*(R**2),
                                   4*(np.pi**2)*(R**2)*np.sqrt((1-nCoinc/nR)*nCoinc/nR**2),
                                   nAntiCoinc/nR*4*(np.pi**2)*(R**2),
                                   4*(np.pi**2)*(R**2)*np.sqrt((1-nAntiCoinc/nR)*nAntiCoinc/nR**2)]
    except:
        pass

# Fill all the NaN to zeros
# Data.fillna(0)
# DataGF.fillna(0)

Finished processing data files for 1000 keV
Finished processing data files for 1125 keV
Finished processing data files for 1266 keV
Finished processing data files for 1425 keV
Finished processing data files for 1603 keV
Finished processing data files for 1804 keV
Finished processing data files for 2030 keV
Finished processing data files for 2285 keV
Finished processing data files for 2571 keV
Finished processing data files for 2894 keV
Finished processing data files for 3257 keV
Finished processing data files for 3665 keV
Finished processing data files for 4124 keV
Finished processing data files for 4641 keV
Finished processing data files for 5223 keV
Finished processing data files for 5878 keV
Finished processing data files for 6614 keV
Finished processing data files for 7443 keV
Finished processing data files for 8376 keV
Finished processing data files for 9426 keV
Finished processing data files for 10608 keV
Finished processing data files for 11937 keV
Finished processing data files

In [ ]:
# Print the data file statistics
print '\n', Data

In [ ]:
# Print the geometric factors
print '\n', DataGF

# Save the geometry factors to a *.csv file
DataGF.to_csv(DataFolder + 'Geometric_Factor_Electrons.csv', index=True)

In [ ]:
print 'Energy [keV]','Total Error [%]','Coinc Error [%]','Anti-Coinc Error [%]'
for E, err_GF_Total, err_GF_Coinc, err_GF_AntiCoinc in zip(InputEnergies, 
                                       DataGF['GF Total [cm^2 sr] Error']/DataGF['GF Total [cm^2 sr]']*100,
                                       DataGF['GF Coinc [cm^2 sr] Error']/DataGF['GF Coinc [cm^2 sr]']*100,
                                       DataGF['GF Anti-Coinc [cm^2 sr] Error']/DataGF['GF Anti-Coinc [cm^2 sr]']*100):
    print E,'\t\t', np.around(err_GF_Total,3), '\t\t', np.around(err_GF_Coinc,3), '\t\t', np.around(err_GF_AntiCoinc,3)

In [ ]:
##########################################################################################
# Plot the Geometry Factor over a range of particle energies
figG, axG = plt.subplots(1,1)

# Plot data
plt.loglog(DataGF.index/1000., DataGF['GF Coinc [cm^2 sr]'], 'k-', linewidth=1.5, label="Coincidence")
plt.loglog(DataGF.index/1000., DataGF['GF Anti-Coinc [cm^2 sr]'], 'k--', linewidth=1.5, label="Anti-Coincidence")

# Plot uncertainies
# plt.fill_between((DataGF.index/1000.).values,
#                  (DataGF['GF Coinc [cm^2 sr]'] - DataGF['GF Coinc [cm^2 sr] Error']).values.astype(float), 
#                  (DataGF['GF Coinc [cm^2 sr]'] + DataGF['GF Coinc [cm^2 sr] Error']).values.astype(float),
#                  lw=0.,
#                  facecolor='red', 
#                  alpha=0.8, 
#                  interpolate=True)

# plt.fill_between((DataGF.index/1000.).values,
#                  (DataGF['GF Anti-Coinc [cm^2 sr]'] - DataGF['GF Anti-Coinc [cm^2 sr] Error']).values.astype(float), 
#                  (DataGF['GF Anti-Coinc [cm^2 sr]'] + DataGF['GF Anti-Coinc [cm^2 sr] Error']).values.astype(float),
#                  lw=0.,
#                  facecolor='blue', 
#                  alpha=0.8, 
#                  interpolate=True)

# Figure Properties
axG.set_ylabel(r'Geometric Factor (cm$^2$ sr)')  
axG.set_xlabel('Incident Electron Energy (MeV)') 
axG.set_ylim(0.001, 300.)
axG.set_xlim(1., 100.)

from matplotlib.ticker import ScalarFormatter
axG.xaxis.set_major_formatter(ScalarFormatter())

axG.legend(loc='upper left', handlelength=4)

# Tight Layout
plt.tight_layout()

# Save the figure 
file_figG = 'Geometric_Factor_Electrons.pdf'
plt.savefig(DataFolder + file_figG, bbox_inches="tight")
print 'Plot saved to: ' + (DataFolder+file_figG)

# Show the figure
plt.show(figG)

In [ ]:
# Print a LaTeX table using the Pandas DataFrame
print DataGF.to_latex(index=True, columns = ('GF Coinc [cm^2 sr]',
                                             'GF Coinc [cm^2 sr] Error',
                                             'GF Anti-Coinc [cm^2 sr]',
                                             'GF Anti-Coinc [cm^2 sr] Error'),
                                  formatters={'GF Anti-Coinc [cm^2 sr]': lambda x: '{:0.1f}'.format(x),
                                              'GF Anti-Coinc [cm^2 sr] Error': lambda x: '{:0.1f}'.format(x),
                                              'GF Coinc [cm^2 sr]': lambda x: '{:0.1f}'.format(x),
                                              'GF Coinc [cm^2 sr] Error': lambda x: '{:0.1g}'.format(x)})

In [ ]:
# Script to generate Geant4 energy and source particle control 
#for E in (np.around(np.logspace(0, 2.0, num=50),2)*1000):
for E in InputEnergies:
    # Find the largest error between the total, coincidence, and anti-coincidence geometric factors
    errMax = max(DataGF['GF Total [cm^2 sr] Error'].loc[E]/DataGF['GF Total [cm^2 sr]'].loc[E]*100,
                 DataGF['GF Coinc [cm^2 sr] Error'].loc[E]/DataGF['GF Coinc [cm^2 sr]'].loc[E]*100,
                 DataGF['GF Anti-Coinc [cm^2 sr] Error'].loc[E]/DataGF['GF Anti-Coinc [cm^2 sr]'].loc[E]*100)
    
    # If the maximum error is below 1%, scale down the number of source particles (Nr) to save computation time
    # Otherwise scale it up so that we achieve 1% uncertainty
    N_R = np.array(1E6)
    if errMax > 1:
        N_R = np.around(Data['Source Particles'].loc[E]*errMax**2, decimals=-4)
        if N_R > 1E9:
            N_R = np.array(1E9)
        print '/control/alias N_R', N_R.astype(int)
    elif pandas.isnull(errMax):
        print '/control/alias N_R', N_R.astype(int)
    else:
        N_R = np.around(Data['Source Particles'].loc[E]*errMax**2, decimals=-4)
        print '/control/alias N_R', N_R.astype(int)
    
    # Print the energy line
    print "/control/loop runElectron_ISO.mac Ekin", E, E